Recurrent NN - от слова повторяющийся. Если в простой нейронной сети идет такая схема обучения 

1) Данные поступают на вход


2) Функция активации


3) Выходные даннее

То в повторяющейся нейронной сети обучение происходит таким образом

1) Данные поступают на вход


2) Функция активации


3) Выходные данные


4) Изначальные входные данные + выходные данные поступают на вход в функцию активации

Для простой нейронной сети предложение "Ян управляет машиной" тоже самое, что и "Машина управляет Яном". Повторяющиеся нейронные сети помогают в случае, если нам нужна последовательность.

Внутри RNN есть LSTM-"клетки" отвечающие за сохранение знания о последовательности. Внутри LSTM есть концепт Keep Gate и Forget Gate. Так как в RNN постоянно повторяются данные в функции активации из предыдущего выхода функции активации, то NN не нужно запоминать все данные. Иначе получалось бы переобучение и слишком большая нагрузка на модель. Forget Gate - это то, что модель забывает и Keep Gate - это то, что модель забывает. 


LSTM - long short term memory

Основная идея recurrent NN в том, что порядок данных, получаемых на вход имеет значение. К примеру временные ряды, язык (лингвистика, порядок слов в предложении имеет значение).  

https://pythonprogramming.net/recurrent-neural-network-deep-learning-python-tensorflow-keras/

http://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [177]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
#Больше не нужно импортировать CuDNNLSTM, теперь можно просто инициализировать LSTM без функции активации
import pandas as pd
import time
from sklearn import preprocessing
from collections import deque
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
#BatchNormalization -  та же номрализация, но между слоями. По той же самой причине по котрой мы нормализиуем данные для
#инпута, мы можем нормализировать данные для входа в новый слой
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
#ModelCheckpoint - функция в которую мы можем передать аргументы, сообщающие модели, когда сохранять чекпоинты
#К примеру, каждый раз когда мы модель достигает минимумам функции потерь, то мы можем сделать чекпоинт, по сколкьо в течении
#100 эпох модель может переобучиться и под конец функция потерь возрастет, но с помощью чекпоинта мы сохраняем модель,
#которая достигла максимально низкой функции потерь. Конечно, можно сохрнатяь каждую эпоху, однако, понадобится много места

In [178]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#physd = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physd[0], True)

Num GPUs Available:  1


In [179]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.99)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [180]:
mnist = tf.keras.datasets.mnist  # mnist is a dataset of 28x28 images of handwritten digits and their labels
(x_train, y_train),(x_test, y_test) = mnist.load_data()  # unpacks images to x_train/x_test and labels to y_train/y_test

x_train = x_train/255.0
x_test = x_test/255.0

print(x_train.shape)
print(x_train[0].shape)

(60000, 28, 28)
(28, 28)


Что собой представляем MNIST - это 28 строк с 28 ячейкамис  данными о пикселях. Нам важно то в каком порядке идут пиксели, поэтому нам нужен LSTM

Нормализуем данные:

In [181]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [182]:
model = Sequential() 

#назначим модель с LSTM слоем, 128 нейронов, назначаем параметр return_sequences, чтобы учитывалась последовательность
model.add(LSTM(128, input_shape=(x_train.shape[1:]), return_sequences=True)) 
model.add(Dropout(0.2)) 

model.add(LSTM(128)) 
model.add(Dropout(0.2))
          
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
          
model.add(Dense(10, activation='softmax')) #последним слой, 10 - это количество классов         

In [183]:
# opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5) #настроим оптимайзер
# #пр. 1e-5 = 0.00001

# model.compile(
#     loss='sparse_categorical_crossentropy',
#     optimizer=opt,
#     metrics=['accuracy'],
# )

# model.fit(x_train,
#           y_train,
#           epochs=3,
#           validation_data=(x_test, y_test))

## Cryptocurrency-predicting RNN

Будем анализировать значение e-валюты

В данном деле нужно придать большое значение работе с временными рядами и предобработке данных

In [205]:
#LTC-USD значение Light Coin в долларах и т.д.
#time - время по UTC, 
#low-самая низкая цена за период минуты
#High-самая высокая цена за период минуты
#open-цена в начале минуты
#close-цена в конце минуты
#volume - доля в общей прибыли рынка

In [206]:
SEQ_LEN = 60 #мы будем использовать 60 периодов (здесь один период - одна минута), чтобы предсказать данные 
FUTURE_PERIOD_PREDICT = 3 #на какой период предсказываем (здесь один период - одна минута), и мы выбираем предсказывать
# за 3 минуты до
RATIO_TO_PREDICT = 'LTC-USD' #для какой валюты строим модель
EPOCHS = 10 #сколько эпох мы хотим, чтобы модель училась
BATCH_SIZE = 64 #сколько батчей
NAME = f"{RATIO_TO_PREDICT}-RATIO-{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}" #даем модели имя, чтобы ориентироваться в tensorboard

In [207]:
#создадим функцию для создания таргета
def classify(current, future):
    #current - текущая цена
    #future - будующая цена
    #здесь логика для модели обучения
    #если будущая цена выше текущей, то
    if float(future) > float(current):
        return 1 #это хорошая цена и нам стоит купить валюту
    else:
        return 0 #это плохая цена и нам не стоит покупать валюту

In [208]:
# инициализируем и соединим датафреймы
#нам нужным только значения volume и close

main_df = pd.DataFrame()

ratios = ["BTC-USD", "LTC-USD", 'ETH-USD', 'BCH-USD']
for ratio in ratios:
    dataset = f"crypto_data/crypto_data/{ratio}.csv" 
    df = pd.read_csv(dataset, names = ['time', 'low', 'high', 'open', 'close','volume'])
    
    df.rename(columns={"close" : f"{ratio}_close", "volume" : f"{ratio}_volume"}, inplace=True)
    
    df.set_index('time', inplace=True)
    
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    if len(main_df) == 0:
        main_df = df
    else: 
        main_df = main_df.join(df)

In [209]:
main_df.head()

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume
time,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361
1528968720,6487.379883,7.706374,96.660004,314.387024,486.01001,26.019083,870.859985,26.856577
1528968780,6479.410156,3.088252,96.570000,77.129799,486.00000,8.449400,870.099976,1.124300
1528968840,6479.410156,1.404100,96.500000,7.216067,485.75000,26.994646,870.789978,1.749862
1528968900,6479.979980,0.753000,96.389999,524.539978,486.00000,77.355759,870.000000,1.680500


In [210]:
#создадим колонку, которая покажет сколько валюта будет стоить через 3 минуты
#future заивисит от колонки close
main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT) 

In [211]:
#создадим таргет-колонку
main_df['target'] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df['future']))

In [212]:
#оценим работу функции
main_df[[f"{RATIO_TO_PREDICT}_close",'future','target']].head(10)

,LTC-USD_close,future,target
time,,,
1528968660,96.580002,96.500000,0
1528968720,96.660004,96.389999,0
1528968780,96.570000,96.519997,0
1528968840,96.500000,96.440002,0
1528968900,96.389999,96.470001,1
1528968960,96.519997,96.400002,0
1528969020,96.440002,96.400002,0
1528969080,96.470001,96.400002,0
1528969140,96.400002,96.400002,0


In [213]:
#на всякий случай отсортируем данные по времени 
times = sorted(main_df.index.values) 

In [214]:
#отделим последние 5% времени, которые будем использовать проверки (валидации)
last_5pct = times[-int(0.05*len(times))] #создадим порог, который покажет число, находящееся на границе 95% времени всего df
validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

In [215]:
#создадим функцию для предобработки df (функцию нужна, т.к. нам нужно будет сделать тоже самое и с валидационным df) 

In [216]:
main_df.head()

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume,future,target
time,,,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361,96.500000,0
1528968720,6487.379883,7.706374,96.660004,314.387024,486.01001,26.019083,870.859985,26.856577,96.389999,0
1528968780,6479.410156,3.088252,96.570000,77.129799,486.00000,8.449400,870.099976,1.124300,96.519997,0
1528968840,6479.410156,1.404100,96.500000,7.216067,485.75000,26.994646,870.789978,1.749862,96.440002,0
1528968900,6479.979980,0.753000,96.389999,524.539978,486.00000,77.355759,870.000000,1.680500,96.470001,1


In [217]:
def preprocess_df(df):
    df = df.drop('future', axis=1) #она просто больше не нужна (и она даст огромное понятие сети как предсказывать таргет)
    
    for col in df.columns:
        if col != 'target': #для всех колонок кроме колонки с таргетом
            #pct_change() - показывает на сколько процентов изменился текущий элемент по сравнению с предыдущим
            #пр. текущий 5, предыдущий 3. 1-(6/3) = 1. Значит текущий элемент возрос на 100% по сравнению с предыдущим
            #таким образом мы нормализируем данные, также у каждой валюты свое значение в usd, различающееся в размере
            #Нам интересны не сами значения валют в долларах для признаков, но именно магнитуда, тренд. Таким образом мы
            #и получаем нужное нам движение
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values) #масштабируем данные
            
    df.dropna(inplace=True) #на случай если pct_change и scale создаст Nans
    
    sequential_data = []    
    #к примеру у нас есть лист в который мы добавляем элементы, но когда этот лист достигнет длины = 60, то как только 
    #мы добавим в лист новый объект, то он выбросит из листа старый объект
    #мы будем ждать, пока prev_days будем иметь 60 значений 
    prev_days = deque(maxlen=SEQ_LEN)
    
    for i in df.values:
        #df.values - конвертирует df в list of list 
        #добавляем list, т.к. sequential_data будет последовательность из lists 
        prev_days.append([n for n in i[:-1]]) # n for n - для каждого элемента в строке кроме последнего элемента в строке
        #который является таргетом и нам не нужен 
        if len(prev_days) == SEQ_LEN:
             #здесь мы будем добавлять X и y
            sequential_data.append([np.array(prev_days), i[-1]]) #добавляем np.array из 60 строк с признакаами и i - таргет
    
    random.shuffle(sequential_data)
    
    #привнесем баланс в таргеты
    buys = [] #лист для таргет == 1
    sells = [] #лист для таргет == 0
    
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
            
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys), len(sells)) #узнамем, где наименьашя длина и будем использовать её, чтобы обе части были равны
    
    buys = buys[:lower]
    sells = sells[:lower]
    
    #совмещаем 
    sequential_data = buys+sells
    random.shuffle(sequential_data)
    
    #сплитим данные на признаки и таргеты
    X = []
    y = []
    
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
        
    return np.array(X).astype('float32'), np.array(y)

In [218]:
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

In [233]:
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {list(train_y).count(0)}, buys: {list(train_y).count(0)}")
print(f"VALIDATION Dont buys: {list(validation_y).count(0)}, buys: {list(validation_y).count(1)}")

train data: 69188 validation: 3062
Dont buys: 34594, buys: 34594
VALIDATION Dont buys: 1531, buys: 1531


Начнем строить модель

In [221]:
model = Sequential()

model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=(train_x.shape[1:]))) #т.к. дальше будет Dense, то убираем параметр return_sequences, т.к.
#он больше не нужен
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax')) #т.к. у нас бинарная классификация, то всего 2 нейрона и softmax

opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = opt,
             metrics = ['accuracy'])

#инициализириуем tensorboard
tensorboard = TensorBoard(log_dir=f'logs/{NAME}')

filepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}" #уникальное имя файла, которое будет состоять из эпохи и метрики acc для val

checkpoint = ModelCheckpoint("models/{}.hd5".format(filepath,
                                                      monitor='val_accuracy',
                                                      verbose=1,
                                                      save_best_only=True,
                                                      mode='max')) # сохраняет лучшую модель

history = model.fit(train_x, train_y,
                   batch_size = BATCH_SIZE,
                   epochs = EPOCHS,
                   validation_data = (validation_x, validation_y),
                   callbacks = [tensorboard, checkpoint])

Epoch 1/10
1081/1082 [============================>.] - ETA: 0s - loss: 0.7141 - accuracy: 0.5215

INFO:tensorflow:Assets written to: models\RNN_Final-01-0.537.hd5\assets


INFO:tensorflow:Assets written to: models\RNN_Final-01-0.537.hd5\assets


1082/1082 [==============================] - 56s 48ms/step - loss: 0.7141 - accuracy: 0.5215 - val_loss: 0.6885 - val_accuracy: 0.5372
Epoch 2/10
1082/1082 [==============================] - ETA: 0s - loss: 0.6870 - accuracy: 0.5447

INFO:tensorflow:Assets written to: models\RNN_Final-02-0.500.hd5\assets


...

INFO:tensorflow:Assets written to: models\RNN_Final-09-0.554.hd5\assets


1082/1082 [==============================] - 58s 54ms/step - loss: 0.6845 - accuracy: 0.5564 - val_loss: 0.6851 - val_accuracy: 0.5539
Epoch 10/10
1082/1082 [==============================] - ETA: 0s - loss: 0.6827 - accuracy: 0.5615

INFO:tensorflow:Assets written to: models\RNN_Final-10-0.562.hd5\assets


INFO:tensorflow:Assets written to: models\RNN_Final-10-0.562.hd5\assets


1082/1082 [==============================] - 63s 58ms/step - loss: 0.6827 - accuracy: 0.5615 - val_loss: 0.6822 - val_accuracy: 0.5624


Accuracy для трейна может быть ниже accuracy для валидации, т.к. accuracy для трейна складывается из среднего между всеми эпохами.